In [201]:
import os
import re
import random
from string import punctuation
from argparse import Namespace
from collections import defaultdict

import IPython

import torch as th
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm
import torchaudio
import numpy as np

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import sentencepiece as spm

import textgrids
import faiss

from sentence_transformers import SentenceTransformer
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

from fairseq.data import Dictionary
from fairseq.models.mST.w2v2_phone_transformer import W2V2Transformer
from fairseq.data.audio.multilingual_triplet_v2_phone_dataset import (
    MultilingualTripletDataConfig,
    MultilingualTripletDataset,
    MultilingualTripletDatasetCreator
)
from fairseq.data.audio.speech_to_text_dataset import get_features_or_waveform
from examples.speech_to_text.data_utils import load_df_from_tsv, save_df_to_tsv
from fairseq.checkpoint_utils import load_checkpoint_to_cpu
from fairseq.data.encoders.sentencepiece_bpe import SentencepieceBPE, SentencepieceConfig

from fairseq.models.mST.w2v2_phone_transformer import W2V2Transformer
from fairseq.data.audio.multilingual_triplet_v2_phone_dataset import (
    MultilingualTripletDataConfig,
    MultilingualTripletDataset,
    MultilingualTripletDatasetCreator
)

# Build Code-Switch for CV 9.0 Language

In [5]:
root = '/mnt/data/siqiouyang/datasets/covost2/'

# Tokenization

In [6]:
tokenizer = spm.SentencePieceProcessor()
tokenizer.Load('/mnt/data/siqiouyang/runs/mST/pretrained/mbart50.ft.n1/sentence.bpe.model')

True

In [7]:
langs = ['de', 'fr', 'es', '-it-', 'ru', 'zh-CN', 'pt', '-fa-', '-et-', '-mn-', 'nl', 'tr', '-ar-', 'sv-SE', '-lv-', '-sl-', '-ta-', '-ja-', '-id-']

In [286]:
lang = 'sv-SE'
lang_root = os.path.join(root, lang)

In [287]:
lang_df = load_df_from_tsv(os.path.join(lang_root, 'train_st_{}_en.tsv'.format(lang)))

In [288]:
lang_df

,id,audio,n_frames,src_text,tgt_text,speaker,src_lang,tgt_lang
0,common_voice_sv-SE_18747821,common_voice_sv-SE_18747821.wav,49536,Du ska inte flyga!,You should not fly!,2c3ffd78f56f2fefc02b7e5cfd35004f17ff411bacf92f...,sv-SE,en
1,common_voice_sv-SE_18747822,common_voice_sv-SE_18747822.wav,64512,Vi klarar oss tills grödorna har grott.,We’ll be fine until the crops have grown.,2c3ffd78f56f2fefc02b7e5cfd35004f17ff411bacf92f...,sv-SE,en
2,common_voice_sv-SE_18747824,common_voice_sv-SE_18747824.wav,50688,"Nej, jag är bara glad.","No, I’m just happy.",2c3ffd78f56f2fefc02b7e5cfd35004f17ff411bacf92f...,sv-SE,en
3,common_voice_sv-SE_18747826,common_voice_sv-SE_18747826.wav,57600,Jag har inte sökt uppmärksamhet.,I have not been seeking attention.,2c3ffd78f56f2fefc02b7e5cfd35004f17ff411bacf92f...,sv-SE,en
4,common_voice_sv-SE_18748002,common_voice_sv-SE_18748002.wav,61056,För att du inte har betalat?,Because you’ve not paid?,2c3ffd78f56f2fefc02b7e5cfd35004f17ff411bacf92f...,sv-SE,en
...,...,...,...,...,...,...,...,...
2155,common_voice_sv-SE_18786940,common_voice_sv-SE_18786940.wav,59520,"Herregud, visst, men gör det riktigt snofsigt.","My God, sure, but make it really swanky.",d24ffe3638b0365e1b653ea982e91d4b7e76a458994db9...,sv-SE,en
2156,common_voice_sv-SE_18786941,common_voice_sv-SE_18786941.wav,37632,Du får inte ta min Maria.,You mustn’t take my Maria.,d24ffe3638b0365e1b653ea982e91d4b7e76a458994db9...,sv-SE,en
2157,common_voice_sv-SE_18786942,common_voice_sv-SE_18786942.wav,41472,"Nej, för många kalorier.","No, too many calories.",d24ffe3638b0365e1b653ea982e91d4b7e76a458994db9...,sv-SE,en
2158,common_voice_sv-SE_18786943,common_voice_sv-SE_18786943.wav,40320,Det var när de fick bonuslådan.,It was when they got the bonus box.,d24ffe3638b0365e1b653ea982e91d4b7e76a458994db9...,sv-SE,en


In [289]:
sentences = lang_df['src_text'].tolist()
sentences

['Du ska inte flyga!',
 'Vi klarar oss tills grödorna har grott.',
 'Nej, jag är bara glad.',
 'Jag har inte sökt uppmärksamhet.',
 'För att du inte har betalat?',
 'Vi kanske ska lösa in.',
 'Det visade sig att gruvan hade färre äkta diamanter än en ordinär pantlånare.',
 'Se från början.',
 'Paris är så litet.',
 'Därför startade du ert skitföretag.',
 'Vi vet redan att skörden kommer att bli dålig i många områden.',
 'Det här känns som någon smaklös fälla.',
 'Gör det inte.',
 'Jag vet inte vad han håller på med.',
 'Du har rätt i det.',
 'En blek kopia av henne.',
 'Och den nya ägaren kommer att drabbas av olycka.',
 'Lägg ner fågelmetaforerna.',
 'Till och med fångar får en sista måltid.',
 'Och som ägare, var jag skyldig.',
 'Man kan dö för den.',
 'Vem bryr sig?',
 'Det var ett snabbt beslut.',
 'Nej, jag är förbannad.',
 'Har du glömt vår dejt?',
 'Så om vi kopplar ihop några.',
 'Gav du mig alla de klyschorna för detta?',
 'Jag tror att folk matar dem varje dag.',
 'Hon såg di

In [290]:
tokenized_sentences = []
segmentss = []
space = tokenizer.IdToPiece(5)

# specifically for zh-CN
# pattern = ",|\?|・|\u3002|\uff1f|\uff01|\uff0c|\u3001|\uff1b|\uff1a|\u201c|\u201d|\u2018|\u2019|\uff08|\uff09|\u300a|\u300b|\u3008|\u3009|\u3010|\u3011|\u300e|\u300f|\u300c|\u300d|\ufe43|\ufe44|\u3014|\u3015|\u2026|\u2014|\uff5e|\ufe4f|\uffe5" + punctuation
# for sent in sentences:
#     tokens = tokenizer.EncodeAsPieces(sent)
#     segments = []
#     for idx, token in enumerate(tokens):
#         if token != space and token not in pattern:
#             segments.append((idx, idx))
#     tokenized_sentence = []
#     for seg in segments:
#         tokenized_sentence.append(''.join(tokens[seg[0] : seg[1] + 1]).replace(space, ''))

#     segmentss.append(segments)
#     tokenized_sentences.append(tokenized_sentence)

# for other languages
def skip(token, punctuation, space):
    if token.startswith(space):
        token = token[1:]
        if token == '':
            return 1
    for c in token:
        if c not in punctuation:
            return 0
    return 1

# punctuation = '!"#$%&,.:;?'
for sent in tqdm(sentences):
    tokens = tokenizer.EncodeAsPieces(sent)
    segments = []
    last = -1
    for idx, token in enumerate(tokens):
        if token.startswith(space) or token in punctuation:
            if last != -1 and last <= idx - 1:
                segments.append((last, idx - 1))
            last = idx + (token in punctuation)
    if last < len(tokens):
        segments.append((last, len(tokens) - 1))

    tokenized_sentence = []
    for seg in segments:
        tokenized_sentence.append(''.join(tokens[seg[0] : seg[1] + 1]).replace(space, ''))

    tokenized_sentences.append(tokenized_sentence)
    segmentss.append(segments)

100%|██████████| 2160/2160 [00:00<00:00, 60373.02it/s]


In [291]:
print(*tokenized_sentences[10:20], sep='\n')

['Vi', 'vet', 'redan', 'att', 'skörden', 'kommer', 'att', 'bli', 'dålig', 'i', 'många', 'områden']
['Det', 'här', 'känns', 'som', 'någon', 'smaklös', 'fälla']
['Gör', 'det', 'inte']
['Jag', 'vet', 'inte', 'vad', 'han', 'håller', 'på', 'med']
['Du', 'har', 'rätt', 'i', 'det']
['En', 'blek', 'kopia', 'av', 'henne']
['Och', 'den', 'nya', 'ägaren', 'kommer', 'att', 'drabbas', 'av', 'olycka']
['Lägg', 'ner', 'fågelmetaforerna']
['Till', 'och', 'med', 'fångar', 'får', 'en', 'sista', 'måltid']
['Och', 'som', 'ägare', 'var', 'jag', 'skyldig']


In [292]:
print(*sentences[10:20], sep='\n')

Vi vet redan att skörden kommer att bli dålig i många områden.
Det här känns som någon smaklös fälla.
Gör det inte.
Jag vet inte vad han håller på med.
Du har rätt i det.
En blek kopia av henne.
Och den nya ägaren kommer att drabbas av olycka.
Lägg ner fågelmetaforerna.
Till och med fångar får en sista måltid.
Och som ägare, var jag skyldig.


In [293]:
# for i, id in enumerate(tqdm(lang_df['id'])):
#     with open(os.path.join(lang_root, '16kHz', '{}.txt'.format(id)), 'w') as w:
#         w.write(' '.join(tokenized_sentences[i]))

Doing MFA alignment.

In [294]:
filtered_grids = []
n_outlier = 0
for i, id in enumerate(tqdm(lang_df['id'])):
    # grid_path = os.path.join(dict_lang_root, '16kHz/align_wmseg/{}.TextGrid'.format(id))
    grid_path = os.path.join(lang_root, '16kHz/align_spm/{}.TextGrid'.format(id))
    if os.path.exists(grid_path):
        grid = textgrids.TextGrid(grid_path)
        filtered_grid = [tok for tok in grid['words'] if tok.text != '']

        if len(filtered_grid) != len(tokenized_sentences[i]):
            # print([w.text for w in filtered_grid], tokenized_sentences[i], sep='\n')
            n_outlier += 1
            continue

        interval = np.array([(word.xmin, word.xmax) for word in filtered_grid])
        audio_path = os.path.join(lang_root, '16kHz/{}.wav'.format(id))
        info = torchaudio.info(audio_path)
        duration = info.num_frames / info.sample_rate
        interval = interval / duration

        th.save([segmentss[i], interval], os.path.join(lang_root, '16kHz', '{}.pt'.format(id)))

100%|██████████| 2160/2160 [00:00<00:00, 2413.90it/s]


In [295]:
n_outlier / len(lang_df)

0.001388888888888889

In [296]:
n_outlier

3